In [2]:
!pip install langgraph langchain_groq langchain_community -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.2.15
    Uninstalling langgraph-sdk-0.2.15:
      Successfully uninstalled langgraph-sdk-0.2.15
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.4
    Uninstalling langgraph-1.0.4:
      Successfully uninstalled langgraph-1.0.4


In [3]:
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun

api_key=userdata.get('GROQ')
llm = ChatGroq(
            groq_api_key=api_key,
            model="llama-3.3-70b-versatile",
            temperature=0.0
        )
search_tool = DuckDuckGoSearchRun()

## State

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal
from langchain_core.messages import HumanMessage, AIMessage
import os

class ResearchState(TypedDict):
  question: str
  search_query: str
  search_results: list
  answer: str
  needs_more_info: bool
  iteration_count: int
"""question: Stores the original user's question throughout the entire workflow
search_query: Stores the refined/optimized search query generated by the LLM for better search results
search_results: Stores the documents/data retrieved from the search/vector database
answer: Stores the final generated response from the LLM
needs_more_info: Boolean flag that determines if we need to search again or if the answer is complete
iteration_count: Tracks how many times we've looped through the workflow to prevent infinite loops"""


"question: Stores the original user's question throughout the entire workflow\nsearch_query: Stores the refined/optimized search query generated by the LLM for better search results\nsearch_results: Stores the documents/data retrieved from the search/vector database\nanswer: Stores the final generated response from the LLM\nneeds_more_info: Boolean flag that determines if we need to search again or if the answer is complete\niteration_count: Tracks how many times we've looped through the workflow to prevent infinite loops"

## Define the Nodes

In [8]:
def analyze_question(state: ResearchState):
    """Analyzes the question and creates a search query"""
    question = state["question"]
    iteration = state.get("iteration_count", 0)

    # If first iteration, use original question; otherwise refine
    if iteration == 0:
        search_query = question
    else:
        # Refine the query based on previous results
        prompt = f"""
        Original question: {question}
        Previous search didn't give enough info.
        Create a more specific search query to find better results.
        Return only the search query, nothing else.
        """
        response = llm.invoke([HumanMessage(content=prompt)])
        search_query = response.content.strip()

    return {
        "search_query": search_query,
        "iteration_count": state.get("iteration_count", 0) + 1
    }

def search_web(state: ResearchState):
    """Searches for information using DuckDuckGo"""
    query = state["search_query"]

    print(f"🔍 Searching for: {query}")

    try:
        # Perform web search
        results = search_tool.run(query)
        print(f"✅ Found results (length: {len(results)} chars)")
        return {"search_results": results}
    except Exception as e:
        print(f"❌ Search failed: {e}")
        return {"search_results": f"Search failed: {str(e)}"}

def generate_answer(state: ResearchState):
    """Generates answer and checks if it's complete"""
    results = state["search_results"]
    question = state["question"]
    iteration = state["iteration_count"]

    prompt = f"""
    Question: {question}

    Search Results:
    {results}

    Based on the search results above, provide a comprehensive answer to the question.

    Important:
    - If the search results provide enough information, give a complete answer.
    - If you don't have enough information to fully answer, respond with "NEED_MORE_INFO: [brief explanation of what's missing]"
    - This is iteration {iteration} of 3.
    """

    print(f"🤖 Generating answer (iteration {iteration})...")

    response = llm.invoke([HumanMessage(content=prompt)])
    answer = response.content

    needs_more = "NEED_MORE_INFO" in answer

    if needs_more:
        print(f"⚠️  Answer incomplete, needs more research")
    else:
        print(f"✅ Answer complete!")

    return {
        "answer": answer,
        "needs_more_info": needs_more
    }

In [9]:
def should_continue_research(state: ResearchState) -> Literal["search", "end"]:
    """Decides if we need to search more or finish"""
    if state["needs_more_info"] and state["iteration_count"] < 3:
        print(f"🔄 Looping back for more research (iteration {state['iteration_count']}/3)")
        return "search"  # Go back and search again
    else:
        if state["iteration_count"] >= 3:
            print(f"⏹️  Max iterations reached, stopping")
        return "end"  # We're done

## Build the Graph

In [10]:
def create_research_graph():
    """Creates and compiles the research graph"""
    workflow = StateGraph(ResearchState)

    # Add nodes
    workflow.add_node("analyze", analyze_question)
    workflow.add_node("search", search_web)
    workflow.add_node("generate", generate_answer)

    # Add edges
    workflow.set_entry_point("analyze")  # Start here
    workflow.add_edge("analyze", "search")  # analyze → search
    workflow.add_edge("search", "generate")  # search → generate

    # Add conditional edge (the magic!)
    workflow.add_conditional_edges(
        "generate",  # From this node
        should_continue_research,  # Use this function to decide
        {
            "search": "analyze",  # If needs more info, loop back
            "end": END  # Otherwise, finish
        }
    )

    # Compile the graph
    return workflow.compile()



In [14]:
# 5. Main execution
if __name__ == "__main__":
    print("=" * 60)
    print("LangGraph Research Assistant")
    print("=" * 60)

    # Create the graph
    app = create_research_graph()

    # Run with a sample question
    question = "How many episodes relaesed in Onepiece anime and chapters in manga?"

    print(f"\n❓ Question: {question}\n")
    print("-" * 60)

    result = app.invoke({
        "question": question,
        "iteration_count": 0
    })

    print("\n" + "=" * 60)
    print("FINAL ANSWER:")
    print("=" * 60)
    print(result["answer"])
    print("\n" + "=" * 60)
    print(f"Total iterations: {result['iteration_count']}")
    print("=" * 60)

LangGraph Research Assistant

❓ Question: How many episodes relaesed in Onepiece anime and chapters in manga?

------------------------------------------------------------
🔍 Searching for: How many episodes relaesed in Onepiece anime and chapters in manga?
✅ Found results (length: 1237 chars)
🤖 Generating answer (iteration 1)...
⚠️  Answer incomplete, needs more research
🔄 Looping back for more research (iteration 1/3)
🔍 Searching for: "One Piece anime total episodes and manga chapters count"
✅ Found results (length: 760 chars)
🤖 Generating answer (iteration 2)...
⚠️  Answer incomplete, needs more research
🔄 Looping back for more research (iteration 2/3)
🔍 Searching for: One Piece anime total episodes and manga chapters count
✅ Found results (length: 1349 chars)
🤖 Generating answer (iteration 3)...
✅ Answer complete!
⏹️  Max iterations reached, stopping

FINAL ANSWER:
The One Piece anime has released more than 1150 episodes as of November 2025. The manga series, on the other hand, has 